# Packages

In [ ]:
using FinEtools, FinEtoolsAcoustics, LinearAlgebra

# Initial setup

In [ ]:
rho = 1.21 * phun("kg/m^3") # mass density
c  = 343.0 * phun("m/s") # sound speed
bulk =  c ^ 2 * rho
frequency = 500.0 # frequency of the incident wave, Hz
omega = 2 * pi * frequency
dt = 1.0 / frequency / 20
tfinal = 5 * dt
nsteps = round(tfinal / dt) + 1

# Mesh

In [58]:
# list of meshes
Meshes = Array{Tuple{FENodeSet, AbstractFESet}, 1}()
# Q4 discretization of large rectangular domain divisions
push!(Meshes, Q4quadrilateral([0.0 0.0; 1.2 2.6], 12, 26)) # 1
push!(Meshes, Q4quadrilateral([1.6 5.0; 4.05 5.35], 29, 3)) # 2
push!(Meshes, Q4quadrilateral([0.0 2.6; 4.05 5.0], 40, 24)) # 3
push!(Meshes, Q4quadrilateral([1.35 0.0; 4.05 2.6], 27, 26)) # 4
push!(Meshes, Q4quadrilateral([4.25 4.85; 5.25 5.35], 10, 4)) # 5
push!(Meshes, Q4quadrilateral([4.05 3.12; 5.25 4.85], 12, 17)) # 6
push!(Meshes, Q4quadrilateral([4.25 1.2; 5.25 3.12], 10, 18)) # 7
push!(Meshes, Q4quadrilateral([5.25 1.2; 8.0 3.95], 28, 28)) # 8
# merge meshes
fens, outputfes = mergenmeshes(Meshes, 0.03)
# concatenate connectivities
fes = cat(outputfes[5], cat(outputfes[6], cat(outputfes[7], outputfes[8])))
fes = cat(outputfes[1], cat(outputfes[2], cat(outputfes[3], cat(outputfes[4], fes))))

FESetQ4([(1, 2, 15, 14), (14, 15, 28, 27), (27, 28, 41, 40), (40, 41, 54, 53), (53, 54, 67, 66), (66, 67, 80, 79), (79, 80, 93, 92), (92, 93, 106, 105), (105, 106, 119, 118), (118, 119, 132, 131)  …  (3208, 3209, 3238, 3237), (3237, 3238, 3267, 3266), (3266, 3267, 3296, 3295), (3295, 3296, 3325, 3324), (3324, 3325, 3354, 3353), (3353, 3354, 3383, 3382), (3382, 3383, 3411, 3410), (3410, 3411, 3439, 3438), (3438, 3439, 3467, 3466), (3466, 3467, 3495, 3494)], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], nothing)

# Subdomain selection

In [49]:
# Identify boundary finite element set
bfes = meshboundary(fes)
# In case there are any unconnected nodes, remove them, and renumber the elements.
connected = findunconnnodes(fens, fes)
fens, new_numbering = compactnodes(fens, connected);
fess = renumberconn!(fes, new_numbering);
# The geometry and the solution (pressure) fields
geom = NodalField(fens.xyz)
P = NodalField(zeros(FCplxFlt, size(fens.xyz, 1), 1))

NodalField{ComplexF64}(ComplexF64[0.0 + 0.0im; 0.0 + 0.0im; … ; 0.0 + 0.0im; 0.0 + 0.0im;;], [0; 0; … ; 0; 0;;], Bool[0; 0; … ; 0; 0;;], ComplexF64[0.0 + 0.0im; 0.0 + 0.0im; … ; 0.0 + 0.0im; 0.0 + 0.0im;;], 0)

# Visualize geometry

In [60]:
# Export three VTK files: one for the interior of the fluid, and one for the boundary
vtkexportmesh("interior.vtk", fes.conn, fens.xyz, FinEtools.MeshExportModule.VTK.Q4)

true

# Setup discrete model

In [61]:
# Number the degrees of freedom in the pressure field.
numberdofs!(P)
# Create the finite element machine for the fluid.
material = MatAcoustFluid(bulk, rho)
femm  =  FEMMAcoust(IntegDomain(fes, GaussRule(3, 2)), material)
# Use the machine calculate the acoustic stiffness and mass matrices.
S  =  acousticstiffness(femm, geom, P)
C  =  acousticmass(femm, geom, P)
# harmonic point load
pointLoad(dpdn, xyz, J, label, t)::Float32 = dpdn[1] = -rho * 0.02 * sin(omega * t)

UndefVarError: UndefVarError: louter not defined

# Time stepping

In [ ]:
# Solve the transient acoustics equations.
# Refer to [1] for details of the formulation.
# The loop executes inside this local scope
P1 = let
    P0 = deepcopy(P)
    P0.values .= 0.0 # initially all pressure is zero
    vP0 = gathersysvec(P0)
    vQ0 = zeros(eltype(vP0), size(vP0))
    # The `P1` field will be the output of this computation:
    # the final value of the pressure field
    P1 = deepcopy(P0)
    t = 0.0 # Initial time
    fi  =  ForceIntensity( # Initial load
      FCplxFlt, 1, (dpdn, xyz, J, label) -> pointLoad(dpdn, xyz, J, label, t)
    )
    # This is the coefficient matrix that needs to be used in the solver. We are
    # not being very careful here to save on computation: it might be best to
    # factorize this matrix, and then use backward and forward solves inside
    # the loop.
    A = (2.0 / dt) * S + D + (dt / 2.0) * C
    step = 0
    while t <= tfinal
        step += 1
        println("Time $t ($(step)/$(round(tfinal / dt) + 1))")
        t += dt
        # Update load
        fi  = ForceIntensity(FCplxFlt, 1, (dpdn, xyz, J, label) -> pointLoad(dpdn, xyz, J, label, t))
        # Solve for the rate of the pressure
        vQ1 = A \ ((2 / dt) * (S * vQ0) - D * vQ0 - C * (2 * vP0 + (dt / 2) * vQ0) + La0 + La1)
        
        # Update the value of the pressure
        vP1 = vP0 + (dt / 2) * (vQ0 + vQ1)
        
        # Swap variables for next step  
        vP0 = deepcopy(vP1)
        vQ0 = deepcopy(vQ1)
        P1 = scattersysvec!(P1, vec(vP1))
        P0 = deepcopy(P1)
        La0 = deepcopy(La1)
    end
    P1 # Return the final pressure
end

# Visualize

In [ ]:
File  =   "sphere_dipole_P1.vtk"
vtkexportmesh(File, fes.conn, geom.values, FinEtools.MeshExportModule.VTK.H8; scalars = [( "realP", real.(P1.values)),])